In [1]:
!pip install transformers[torch]
!pip install accelerate -U
!pip install datasets[audio]
!pip install evaluate
!pip install jiwer
!pip install datasets --upgrade
!pip install rouge

In [2]:
import numpy as np
import IPython.display as ipd
import torch
import transformers
import datasets
import librosa
import re
import json
import random
import matplotlib.pyplot as plt
import pandas as pd
import evaluate
import nltk
import librosa.display

from IPython.display import display as ipd
from IPython.display import HTML
from datasets import load_dataset, ClassLabel, Audio # Remove load_metric from here
from evaluate import load # Import load from evaluate instead
from transformers import WhisperConfig, WhisperProcessor, WhisperFeatureExtractor, WhisperTokenizer, WhisperForConditionalGeneration
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding, DataCollator
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, Trainer, TrainingArguments
from transformers import pipeline
from transformers.utils import send_example_telemetry

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from transformers import AdamW
from datasets import DatasetDict
from datasets import concatenate_datasets

from rouge import Rouge

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
!huggingface-cli download PolyAI/minds14 --repo-type dataset --revision refs/convert/parquet --local-dir . --local-dir-use-symlinks False  --include 'en-US/*'

minds_enUS = load_dataset('./en-US', split="train")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/download.py:132: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
Fetching 1 files: 100% 1/1 [00:00<00:00, 1352.56it/s]
/content


In [4]:
print(minds_enUS)

Dataset({
    features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
    num_rows: 563
})


In [5]:
merged_dataset = concatenate_datasets([minds_enUS])

print(merged_dataset)

Dataset({
    features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
    num_rows: 563
})


In [76]:
intent_classes = [
    {'index': 0, 'name': 'abroad'},
    {'index': 1, 'name': 'adress'},
    {'index': 2, 'name': 'app_error'},
    {'index': 3, 'name': 'atm_limit'},
    {'index': 4, 'name': 'balance'},
    {'index': 5, 'name': 'business_loan'},
    {'index': 6, 'name': 'card_issues'},
    {'index': 7, 'name': 'cash_deposite'},
    {'index': 8, 'name': 'direct_debit'},
    {'index': 9, 'name': 'freeze'},
    {'index': 10, 'name': 'latest_transactions'},
    {'index': 11, 'name': 'joint_account'},
    {'index': 12, 'name': 'high_value_payment'},
    {'index': 13, 'name': 'pay_bill'}
]

In [6]:
def count_feature_lengths(dataset):
    feature_lengths = {}

    for feature in dataset.features:
        values = dataset[feature]
        lengths = [len(str(value)) if not isinstance(value, (int, float, dict, list)) else 1 for value in values]
        feature_lengths[feature] = lengths
    return feature_lengths

# Count the lengths and store them
feature_lengths = count_feature_lengths(merged_dataset)

# Print the lengths for each feature
for feature, lengths in feature_lengths.items():
    print(f"{feature}: {len(lengths)}")

path: 563
audio: 563
transcription: 563
english_transcription: 563
intent_class: 563
lang_id: 563


In [7]:
# Count the lengths and store them
feature_lengths = count_feature_lengths(minds_enUS)

# Print the lengths for each feature
for feature, lengths in feature_lengths.items():
    print(f"{feature}: {len(lengths)}")

path: 563
audio: 563
transcription: 563
english_transcription: 563
intent_class: 563
lang_id: 563


In [8]:
# prompt: delete the english_transcription and lang_id in dataset

# Assuming 'merged_dataset' is defined as in your previous code

def remove_columns(example):
  del example["english_transcription"]
  del example["lang_id"]
  return example

dataset = merged_dataset.map(remove_columns)

dataset

Dataset({
    features: ['path', 'audio', 'transcription', 'intent_class'],
    num_rows: 563
})

In [10]:
dataset[1]

{'path': '/storage/hf-datasets-cache/all/datasets/51125457981586-config-parquet-and-info-PolyAI-minds14-efce24e3/downloads/extracted/cfd42a9443ffb9548ee39e3c64f8b512ca72b9ce5e2ea6b981d44ba6c7265ae8/en-US~JOINT_ACCOUNT/602baf24bb1e6d0fbce922a7.wav',
 'audio': {'path': '602baf24bb1e6d0fbce922a7.wav',
  'array': array([0.        , 0.00024414, 0.        , ..., 0.01306152, 0.01208496,
         0.01135254]),
  'sampling_rate': 8000},
 'transcription': 'Henry County set up a joint account with my wife and where are they at',
 'intent_class': 11}

In [11]:
# prompt: split the data in merged_dataset into 20% for testing

from datasets import DatasetDict, concatenate_datasets

# Assuming 'merged_dataset' is defined as in your previous code

dataset = DatasetDict({'test': dataset.train_test_split(test_size=0.2)['test']
})

dataset

DatasetDict({
    test: Dataset({
        features: ['path', 'audio', 'transcription', 'intent_class'],
        num_rows: 113
    })
})

# testing

# EVALUATE

In [12]:
whisper = 'kairaamilanii/whisper-mind14-enUS'

tokenizer = WhisperTokenizer.from_pretrained(whisper, language="english")
feature_extractor = WhisperFeatureExtractor.from_pretrained(whisper)
processor = WhisperProcessor(
    feature_extractor=feature_extractor,
    tokenizer=tokenizer,
    )

model = WhisperForConditionalGeneration.from_pretrained(whisper)
model.config.forced_decoder_ids = None


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [13]:
metric_wer = evaluate.load("wer")

In [14]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import SmoothingFunction
# from nltk.translate import rouge_score
from rouge import Rouge
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import f1_score, r2_score

normalizer = BasicTextNormalizer()
smoother = SmoothingFunction()

rouge_score = Rouge()

def compute_f1(preds, labels):
    # Convert predictions and labels to binary (0 or 1)
    preds_binary = [1 if p == l else 0 for p, l in zip(preds, labels)]
    labels_binary = [1 if l != processor.tokenizer.pad_token_id else 0 for l in labels]  # Assuming padding token ID is used for no label

    return f1_score(labels_binary, preds_binary)

def compute_r2(preds, labels):
    # try:
    #     # Convert strings to floating-point numbers
    #     preds_float = np.array([float(p) for p in preds])
    #     labels_float = np.array([float(l) for l in labels])

    #     # Calculate R2 score
    #     return r2_score(labels_float, preds_float)
    # except ValueError as e:
    #     print(f"Error: {e}")
    #     return None
    return r2_score(labels, preds)

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    # Word Error Rate (WER)
    wer_ortho = 100 * metric_wer.compute(predictions=pred_str, references=label_str)

    # Normalized WER
    pred_str_norm = [normalizer(pred) for pred in pred_str]
    label_str_norm = [normalizer(label) for label in label_str]

    wer = metric_wer.compute(predictions=pred_str_norm, references=label_str_norm)

    # Accuracy
    correct_preds = sum(1 for p, l in zip(pred_str_norm, label_str_norm) if p == l)
    total_preds = len(pred_str_norm)
    accuracy = correct_preds / total_preds

    # ROUGE Metrics
    rouge_scores = rouge_score.get_scores(pred_str_norm, label_str_norm, avg=True)

    # F1 Score, R2,
    # (Assuming you have the necessary functions or metrics for these calculations)
    f1_score = compute_f1(pred_str_norm, label_str_norm)
    r2_score = compute_r2(pred_ids, label_ids)

    # BLEU Score
    references = [[ref.split()] for ref in label_str_norm]
    predictions = [pred.split() for pred in pred_str_norm]
    bleu_score = corpus_bleu(references, predictions, smoothing_function=smoother.method1)

    return {
        "wer_ortho": wer_ortho,
        "wer": wer,
        "accuracy": accuracy,
        "f1_score": f1_score,
        "r2_score": r2_score,
        "bleu_score": bleu_score,
        "Rouge 1 r": rouge_scores['rouge-1']['r'],
        "Rouge 1 p": rouge_scores['rouge-1']['p'],
        "Rouge 1 f": rouge_scores['rouge-1']['f'],
    }


In [15]:
from functools import partial

model.config.use_cache = False
model.generate = partial(
    model.generate,
    language="english",
    task="transcribe",
    use_cache=True,
)

In [16]:
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
smoother = SmoothingFunction()

In [17]:
audio_file = dataset['test']['audio'][0]
target = dataset['test']['transcription'][0]
device = "cuda:0" if torch.cuda.is_available() else "cpu"

transcriber = pipeline(
    "automatic-speech-recognition",
    model="kairaamilanii/whisper-mind14-enUS",
    chunk_length_s=30,
    device=device,
)


In [20]:
dataset['test'][2]

{'path': '/storage/hf-datasets-cache/all/datasets/51125457981586-config-parquet-and-info-PolyAI-minds14-efce24e3/downloads/extracted/cfd42a9443ffb9548ee39e3c64f8b512ca72b9ce5e2ea6b981d44ba6c7265ae8/en-US~LATEST_TRANSACTIONS/602baf95963e11ccd901ce4a.wav',
 'audio': {'path': '602baf95963e11ccd901ce4a.wav',
  'array': array([ 0.        ,  0.        ,  0.        , ...,  0.        ,
         -0.00024414,  0.00024414]),
  'sampling_rate': 8000},
 'transcription': "hello this is Robert I have two transactions on my account which I'm not I don't recognize the last five or six transactions in the last 72 hours",
 'intent_class': 12}

In [21]:
dataset['test'][1]

{'path': '/storage/hf-datasets-cache/all/datasets/51125457981586-config-parquet-and-info-PolyAI-minds14-efce24e3/downloads/extracted/cfd42a9443ffb9548ee39e3c64f8b512ca72b9ce5e2ea6b981d44ba6c7265ae8/en-US~FREEZE/602ba47f05f96973d679442e.wav',
 'audio': {'path': '602ba47f05f96973d679442e.wav',
  'array': array([ 0.        ,  0.        , -0.00024414, ...,  0.        ,
          0.00024414,  0.        ]),
  'sampling_rate': 8000},
 'transcription': 'can I freeze my card',
 'intent_class': 9}

In [22]:
prediction = transcriber(
    audio_file,
    batch_size=8,
    return_timestamps=True,
)['chunks']

print("timestamp: ", prediction[0]['timestamp'])
print("predict: ", prediction[0]['text'])
print("target: ", target)

/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, 50259], [2, 50359], [3, 50363]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


timestamp:  (0.0, None)
predict:  show me my account balance please
target:  show me my account balance please


In [26]:
def predict_eval(audio_file, target, idx):
    preds = transcriber(
        audio_file,
        batch_size=8,
    )['text']

    # print("preds: ", preds)
    # print("target: ", target)

    pred_words = preds.split(" ")
    ref_words = target.split(" ")

    # Remove any empty strings from the end of the lists
    if pred_words[-1] == '':
        pred_words = pred_words[:-1]
    if ref_words[-1] == '':
        ref_words = ref_words[:-1]

    # Pad the shorter sequence with empty strings
    max_length = max(len(pred_words), len(ref_words))
    pred_words += ['PAD'] * (max_length - len(pred_words))
    ref_words += ['PAD'] * (max_length - len(ref_words))

    # encoded_text = processor(preds, text=target, sampling_rate=16000)

    # print(len(pred_words))
    # print(len(ref_words))
    # print(pred_words)
    # print(ref_words)

    # Check if both prediction and reference are non-empty
    if pred_words and ref_words:
        wer = metric_wer.compute(predictions=pred_words, references=ref_words)
        wer_percentage = wer * 100
    else:
        wer_percentage = 0.0  # or any other default value

    # Compute accuracy
    correct = sum(1 for p, r in zip(pred_words, ref_words) if p == r)
    accuracy = (correct / len(pred_words)) * 100

    # Compute ROUGE Metrics
    rouge_scores = rouge_score.get_scores(preds, target, avg=True)

    f1 = compute_f1(pred_words, ref_words)

    # BLEU Score
    references = [[ref.split()] for ref in ref_words]
    predictions = [pred.split() for pred in pred_words]
    bleu = corpus_bleu(references, predictions, smoothing_function=smoother.method1)
    return {
        "num": idx,
        "wer": wer,
        "werotho": f"{wer_percentage:.2f}%",
        "accuracy": accuracy,
        "f1_score": f1,
        "bleu_score": bleu,
        "Rouge 1 r": rouge_scores['rouge-1']['r'],
        "Rouge 1 p": rouge_scores['rouge-1']['p'],
        "Rouge 1 f": rouge_scores['rouge-1']['f'],
        "predict": preds,
        "target": target,
        "intent": dataset['test']['intent_class'][idx]
    }



preds_eval_list = []

for idx in range(len(dataset['test'])):

  predicted = predict_eval(dataset['test']['audio'][idx], dataset['test']['transcription'][idx], idx)
  preds_eval_list.append(predicted)

/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is

In [27]:
# prompt: save all the result from testing

import pandas as pd

# Assuming preds_eval_list is already populated from the previous code
df = pd.DataFrame(preds_eval_list)

# Save the DataFrame to a CSV file
df.to_csv('evaluation_results_fix.csv', index=False)

# Download the CSV file (requires the files module from google.colab)
from google.colab import files
files.download('evaluation_results_fix.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [28]:
result = pd.read_csv("/content/evaluation_results_fix.csv")
result

,num,wer,werotho,accuracy,f1_score,bleu_score,Rouge 1 r,Rouge 1 p,Rouge 1 f,predict,target,intent
0,0,0.000000,0.00%,100.000000,1.000000,0.046386,1.000000,1.000000,1.000000,show me my account balance please,show me my account balance please,4
1,1,0.200000,20.00%,80.000000,0.888889,0.050297,0.800000,0.800000,0.800000,can i freeze my card,can I freeze my card,9
2,2,0.142857,14.29%,85.714286,0.923077,0.014057,0.875000,0.875000,0.875000,hello this is robert i have two transactions o...,hello this is Robert I have two transactions o...,12
3,3,0.045455,4.55%,95.454545,0.976744,0.017303,0.947368,0.947368,0.947368,hello i just ate only lost my card at the airp...,hello I just ate only lost my card at the airp...,9
4,4,0.000000,0.00%,100.000000,1.000000,0.046386,1.000000,1.000000,1.000000,add someone to my account please,add someone to my account please,11
...,...,...,...,...,...,...,...,...,...,...,...,...
108,108,0.157895,15.79%,84.210526,0.914286,0.011130,0.896552,0.896552,0.896552,hi i'm trying to make a pretty large payment a...,hi I'm trying to make a pretty large payment a...,10
109,109,0.580645,58.06%,41.935484,0.590909,0.004778,0.911765,0.469697,0.620000,hi i'm just calling because the after that usu...,hi I'm just calling because the after that usu...,2
110,110,0.166667,16.67%,83.333333,0.909091,0.044319,0.833333,0.833333,0.833333,i'd like to freeze my card,I'd like to freeze my card,9
111,111,0.000000,0.00%,100.000000,1.000000,0.053183,1.000000,1.000000,1.000000,the office not working correctly,the office not working correctly,2


In [29]:
accuracy_values = [entry['accuracy'] for entry in preds_eval_list]
overall_accuracy = sum(accuracy_values) / len(accuracy_values)

print(f"Overall Accuracy: {overall_accuracy:.2f}%")


Overall Accuracy: 81.78%


In [30]:
wer_values = [entry['wer'] for entry in preds_eval_list]
overall_wer = sum(wer_values) / len(wer_values)

print(f"Overall WER: {overall_wer:.2f}%")

Overall WER: 0.18%


In [31]:
f1_values = [entry["f1_score"] for entry in preds_eval_list]
overall_f1 = sum(f1_values) / len(f1_values)

print(overall_f1)

0.8653056138698154


In [32]:
bleu_values = [entry["bleu_score"] for entry in preds_eval_list]
overall_bleu = sum(bleu_values) / len(bleu_values)

print(overall_bleu)

0.031916957496520286
